In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import konlpy
from konlpy.tag import Okt
from transformers import AutoTokenizer, AutoModel
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

model = load_model('../data/model_train/sa_model.h5') # 훈련시킨 모델 불러오기

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022") # 토크나이저 설정

okt = Okt() # OKT 불러오기

f = open('../data//model_train/stopwords-ko.txt', 'r') # stopwords 불러오기
stopwords = f.read().split('\n')

In [26]:
han = {'banpo':'반포',
       'ddukssum':'뚝섬',
       'gangseo':'강서',
       'gwangnaroo':'광나루',
       'jamsil':'잠실',
       'mangwon':'망원',
       'nanji' : '난지',
       'yanghwa':'양화',
       'yeechon':'이촌',
       'yydo':'여의도'
       }

In [33]:
for i,v in han.items():
    rev = pd.read_csv(f'../data/리뷰_rep/{i}.csv',index_col=0)
    rev_processing = []
    cnt = 0

    for review in rev['리뷰']:
        cnt = cnt +1
        temp_X = []
        temp_X = okt.morphs(review, stem=True)
        temp_X = [word for word in temp_X if not word in stopwords] #불용어 제거
        rev_processing.append(temp_X)

    rev['Processing'] = rev_processing
    # csv 형식으로 저장
    rev.to_csv(f'../data/SA/{v}.csv', encoding = 'utf-8', index=False)

    # 이렇게 다시 불러오면 데이터 프레임 내 Processing 의 형태가 변환된다
    rev = pd.read_csv(f'../data/SA/{v}.csv', encoding = 'utf-8')

    rv_test = []
    for i in rev.Processing:
        rev_tokenized = tokenizer(i, return_token_type_ids=False, return_attention_mask=False)['input_ids']
        rv_test.append(rev_tokenized)

    max_len = max(len(l) for l in rv_test) # 전체 데이터의 길이를 최대 길이로 맞춘다
    rv_test = pad_sequences(rv_test, maxlen=max_len)

    print(f'{v} 파트 모델 예측 진행')
    predict = model.predict(rv_test) # 모델 사용해서 예측
    label_predict = []
    for i in range(len(rev)):
        if np.argmax(predict[i]) == 0:
            label_predict.append('부정')
        elif np.argmax(predict[i]) == 1:
            label_predict.append('중립')
        elif np.argmax(predict[i]) == 2:
            label_predict.append('긍정')
    rev['평가'] = label_predict

    # csv 형식으로 저장
    rev.to_csv(f'../data/SA/{v}_평가.csv', encoding = 'utf-8', index=False)

    print(f'{v} 파트 끝.')


140/140 [==============================] - 10s 64ms/step
반포 파트 끝.
11/11 [==============================] - 1s 61ms/step
뚝섬 파트 끝.
5/5 [==============================] - 0s 50ms/step
강서 파트 끝.
16/16 [==============================] - 1s 40ms/step
광나루 파트 끝.
78/78 [==============================] - 4s 47ms/step
잠실 파트 끝.
98/98 [==============================] - 7s 66ms/step
망원 파트 끝.
25/25 [==============================] - 2s 64ms/step
난지 파트 끝.
26/26 [==============================] - 1s 38ms/step
양화 파트 끝.
19/19 [==============================] - 1s 29ms/step
이촌 파트 끝.
125/125 [==============================] - 7s 58ms/step
여의도 파트 끝.


## 위를 통해 현재 보유한 리뷰 텍스트에 대한 감성분석을 모두 끝냄


In [44]:
import glob
df = pd.DataFrame()
f = glob.glob('../data/SA/*_평가.csv')
for i in f:
    df1 = pd.read_csv(i,encoding='utf-8')
    plc = i.split('\\')[1].split('_')[0]
    df1['공원'] = plc
    df = pd.concat([df,df1])


df


,리뷰,rep,index,Processing,평가,공원
0,한강 하구에 위치한 공원으로 공원내 생태 습지가 함께하고 있어 일반 쉼터공원으로 서...,0,0,"['한강', '하다', '위치', '한', '공원', '공', '원내', '생태',...",긍정,강서
1,또한 맞은 편에 행주산성이 있고 북동쪽 으로는 북한산이 한눈에 들어오는 전망이 좋은...,1,0,"['맞다', '편', '행주산성', '북동쪽', '으로는', '북한', '산이', ...",긍정,강서
2,김포에서 출발하여 방화대교 / 마곡(철도)대교/가양대교/ 미 개통대교 / 성산대교 ...,0,1,"['김포', '출발', '하다', '방화대교', '/', '마곡', '철도', '대...",부정,강서
3,나의 공적도 여기 어디엔가 있을거 같기도 하고..,1,1,"['공적', '도', '엔', '가다', '하다', '..']",부정,강서
4,제3한강교(반포대교) 라는 대중음악도 생각해보고..,2,1,"['한', '강교', '반포대교', '라는', '대중음악', '도', '생각', '...",긍정,강서
...,...,...,...,...,...,...
2488,로맨틱한 장소,0,1784,"['로맨틱', '한', '장소']",부정,잠실
2489,리버뷰,0,1785,"['리버', '뷰']",긍정,잠실
2490,뛰어난,0,1786,['뛰어나다'],긍정,잠실
2491,내 고향,0,1787,"['내', '고향']",긍정,잠실
